# Mobile App API Security Tester
This notebook demonstrates how to test APIs for common security vulnerabilities, including:
- Weak or missing authentication mechanisms.
- Unsecured API calls using HTTP.
- Sensitive data exposure in API responses or headers.


In [16]:
import requests
from prettytable import PrettyTable
import json
from urllib.parse import urlparse

In [17]:
def check_https(url):
    """
    Checks if the API endpoint uses HTTPS.
    """
    parsed_url = urlparse(url)
    if parsed_url.scheme != 'https':
        return "Insecure (HTTP)"
    return "Secure (HTTPS)"


In [18]:
def test_authentication(url, headers=None):
    """
    Tests for missing or weak authentication tokens.
    Returns the response status and headers.
    """
    try:
        response = requests.get(url, headers=headers, timeout=5)
        if response.status_code == 401:
            return "Unauthorized (Authentication Required)"
        elif response.status_code == 403:
            return "Forbidden (Token Invalid or Insufficient Permissions)"
        return f"Accessible (Status: {response.status_code})"
    except requests.exceptions.RequestException as e:
        return f"Error: {str(e)}"


In [19]:
def detect_sensitive_data(response):
    """
    Scans API response for sensitive information like API keys or PII.
    """
    sensitive_keywords = ['key', 'token', 'password', 'ssn', 'credit', 'card']
    findings = []

    try:
        # Check response JSON
        if response.headers.get('Content-Type') == 'application/json':
            data = response.json()
            for key in data.keys():
                if any(sensitive in key.lower() for sensitive in sensitive_keywords):
                    findings.append(f"Sensitive key detected: {key}")

        # Check headers
        for key, value in response.headers.items():
            if any(sensitive in key.lower() for sensitive in sensitive_keywords):
                findings.append(f"Sensitive header detected: {key}")
    except json.JSONDecodeError:
        pass  # Response is not JSON

    return findings or "No sensitive data detected"


In [20]:
def run_security_tests(url, headers=None):
    """
    Runs all security tests on the given URL.
    """
    results = []

    # Test for HTTPS
    https_status = check_https(url)
    results.append(("HTTPS Check", https_status))

    # Test authentication
    auth_status = test_authentication(url, headers)
    results.append(("Authentication Check", auth_status))

    # Test sensitive data exposure
    try:
        response = requests.get(url, headers=headers, timeout=5)
        sensitive_data = detect_sensitive_data(response)
    except requests.exceptions.RequestException as e:
        sensitive_data = [f"Error: {str(e)}"]
    results.append(("Sensitive Data Check", sensitive_data))

    return results


In [21]:
def display_report(results):
    """
    Displays the test results in a tabular format.
    """
    table = PrettyTable()
    table.field_names = ["Test", "Result"]
    for test, result in results:
        if isinstance(result, list):
            result = "\n".join(result)
        table.add_row([test, result])
    print(table)


In [22]:
# Example API URL
url = "http://example.com/api/test"
headers = {
    "Authorization": "Bearer example_token"
}

# Run tests
results = run_security_tests(url, headers)

# Display report
display_report(results)


+----------------------+----------------------------+
|         Test         |           Result           |
+----------------------+----------------------------+
|     HTTPS Check      |      Insecure (HTTP)       |
| Authentication Check |  Accessible (Status: 404)  |
| Sensitive Data Check | No sensitive data detected |
+----------------------+----------------------------+


In [23]:
import argparse

if __name__ == "__main__":
    parser = argparse.ArgumentParser(description="Mobile App API Security Tester")
    parser.add_argument("url", help="API endpoint URL")
    parser.add_argument("--headers", help="Headers as JSON string", default="{}")
    args = parser.parse_args()

    url = args.url
    headers = json.loads(args.headers)

    results = run_security_tests(url, headers)
    display_report(results)


usage: ipykernel_launcher.py [-h] [--headers HEADERS] url
ipykernel_launcher.py: error: unrecognized arguments: -f


SystemExit: 2

D:\Projects\Master\APISecurityTester\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
{
    "sensitive_keywords": ["key", "token", "password", "ssn", "credit", "card"]
}


In [ ]:
with open('config.json') as f:
    config = json.load(f)
sensitive_keywords = config["sensitive_keywords"]
